In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
from dash import no_update

import plotly.graph_objects as go
import pandas as pd

/var/folders/vk/qtf7k0110x7bb94g6sd3655r0000gn/T/ipykernel_6508/3883421336.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/vk/qtf7k0110x7bb94g6sd3655r0000gn/T/ipykernel_6508/3883421336.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
# PATHs

## The directory where the dune data is stored
DUNE_DATA_PATH = '../dune_data'

## The directory where the cryptopunk data with attributes info is stored
CSV_PATH = '../cp'

## The directory where the three databases are stored
DATABASE_PATH = '../database'

## The directory where the cryptopunk images is stored
PUNK_IMG_PATH='../punk_imgs'

## The directory where the visualization data for each visualization is stored
VIS_DATA_PATH = '../vis_data'

## The directory where the scraped tweets data is stored
TWEET_PATH = '../tweets'

In [20]:
data3_path = '{}/vis3_data.csv'.format(VIS_DATA_PATH)

df3 = pd.read_csv(data3_path, header=0, index_col=0)
df3 = df3[df3['price'] <  5000]
df3 = df3[df3['date'] > '2021-01-01']

df3['img_url'] = df3['punk_id'].apply(lambda x: 'https://www.larvalabs.com/cryptopunks/cryptopunk{}.png'.format(x))
df3['bubble_size'] = df3['bubble_size'].apply(lambda x: x)
df3

,date,price,bubble_size,bubble_color,from,to,punk_id,punk_img,img_url
tx_id,,,,,,,,,
11966,2021-01-02,5.47,3,#F2A359,7106,8017,9340,../punk_imgs/9340.png,https://www.larvalabs.com/cryptopunks/cryptopu...
11994,2021-01-02,5.45,2,#F2A359,7579,6245,4861,../punk_imgs/4861.png,https://www.larvalabs.com/cryptopunks/cryptopu...
12004,2021-01-02,5.30,2,#A4031F,7117,1948,9683,../punk_imgs/9683.png,https://www.larvalabs.com/cryptopunks/cryptopu...
12007,2021-01-02,4.90,2,#F2A359,5887,1864,1743,../punk_imgs/1743.png,https://www.larvalabs.com/cryptopunks/cryptopu...
12008,2021-01-02,4.97,2,#F2A359,921,1864,3410,../punk_imgs/3410.png,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...,...,...
40527,2022-05-04,61.50,3,#DB9065,5456,89,1648,../punk_imgs/1648.png,https://www.larvalabs.com/cryptopunks/cryptopu...
40530,2022-05-05,66.66,3,#F2DC5D,5977,8585,2625,../punk_imgs/2625.png,https://www.larvalabs.com/cryptopunks/cryptopu...
40535,2022-05-05,70.00,2,#DB9065,7348,6009,3314,../punk_imgs/3314.png,https://www.larvalabs.com/cryptopunks/cryptopu...


In [ ]:
fig = go.Figure(data=[
    go.Scatter(
        x=df3["date"],
        y=df3["price"],
        mode="markers",
        marker=dict(
            color=df3["bubble_color"],
            size=df3["bubble_size"],
            line={"color": "#444"},
            reversescale=True,
            sizeref=1,
            sizemode="diameter",
            opacity=0.8,
        )
    )
])


# turn off native plotly.js hover effects - make sure to use
# hoverinfo="none" rather than "skip" which also halts events.

fig.update_traces(hoverinfo="none", hovertemplate=None)

fig.update_layout(
    xaxis=dict(title='Date'),
    yaxis=dict(title='Ether Price'),
    plot_bgcolor='rgba(255,255,255,0.1)'
)

fig.show()

In [23]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph-basic-2", figure=fig, clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
])


@app.callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph-basic-2", "hoverData"),
)

def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]

    df_row = df3.iloc[num]
    img_src = df_row['img_url']
    punk_id = df_row['punk_id']
    from_id = df_row['from']
    to_id = df_row['to']
    if len(desc) > 300:
        desc = desc[:100] + '...'

    children = [
        html.Div([
            html.Img(src=img_src, style={"width": "100%"}),
            html.H2(f"{punk_id}", style={"color": "darkblue"}),
            html.P(f"{from_id}"),
            html.P(f"{to_id}"),
        ], style={'width': '200px', 'white-space': 'normal'})
    ]

    return True, bbox, children

app.run_server(mode='inline')


TypeError: an integer is required (got type NoneType)